study for tracking transfer errors. Send automatic alert if a link between a source and a receiving site has a sustained too high transfer failure rate over a given time inverval. Vary model thresholds to study how this affects the number of alerts being sent.

In [73]:
#import packages etc
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
from elasticsearch import Elasticsearch
matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan
import time
import operator

In [74]:
#declare variables
print("Declaring variables.")

#epoch times of start and end of time interval. 
#first week of february. monday 30. Jan to sunday 5. feb.
#found using epochconverter.com
#epoch time must be in milliseconds
epoch_start = 1485730800000 #jan 30th 00.00.00
epoch_end = 1486335599000 #feb 5th 23.59.59
#epoch_end = 1485820799000 # jan30th 23.59.59 (one day for testing)

loop_time = epoch_start #time variable in loop

#loop time increments . one minute? five minutes?
increment = 5 * 60 * 1000; #milliseconds

#thresholds for defining an error
failure_rate = .9 #failure rate threshold. Q: what is the threshold of red cells in DDM dashboard?
file_thresh = 10 #number of files threshold.
persist_thresh = 15 * 60 * 1000; #milliseconds. time needed before alert is sent.

#number of alarms
alarms = 0



#some kind of dictionary with pairs of sites, and some variable keeping track of time
#if pairs stay on this list too long an alarm is raised {[source,destination]:loop_time}
observed_endpoints = {}

#dict of when an alert was made and what pair is failing {when: [source,receiver]}
print("Done.")

Declaring variables.
Done.


In [75]:
#could be useful to track how much time is spent on this block
time_tracker = time.time()

# algorithm:

print("Starting loop.")
# loop over time interval in small time increments
while loop_time <= epoch_end:
    loop_time = loop_time + increment
    # define search queries for the time increment. one to get transfers overall and one to get failed transfers
    query_done = {
      "size": 0,
      "query": {
        "bool": {
        "must": [
        {
          "range": {
            "@timestamp": {
            "gte": loop_time - increment,
            "lte": loop_time,
            "format": "epoch_millis"
             }
            }
           },
          {"term": {"event_type": "transfer-done"}},
          {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
      }
    }
    query_failed = {
      "size": 0,
      "query": {
        "bool": {
        "must": [
        {
          "range": {
            "@timestamp": {
            "gte": loop_time - increment,
            "lte": loop_time,
            "format": "epoch_millis"
             }
            }
           },
          {"term": {"event_type": "transfer-failed"}},
          {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
      }
    }
    # make scrolls
    es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) 
    my_index = "rucio-events-2017*"
    scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
    scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)
    
    #define once-per-loop variables
    endpoints_failures = {}
    failure_sources = {}
    failure_destinations = {}
    endpoints_transfers = {}
    transfer_sources = {}
    transfer_destinations = {}
    ratios = {}
    counter = 0
    
    
    #loop over transfers and store information about sources and destinations
    for res in scroll_done:
        source = res['_source']['payload']['src-rse']
        destination = res['_source']['payload']['dst-rse']
        pair = (source,destination)
        
        if pair not in endpoints_transfers.keys():
            endpoints_transfers[pair] = 1
        if pair in endpoints_transfers.keys():
            endpoints_transfers[pair] = endpoints_transfers[pair]+1
        
        counter += 1
    
    
    counter = 0 #reset counter
    #loop over failures and store information about sources and destinations
    for res in scroll_errors:
        source = res['_source']['payload']['src-rse']
        destination = res['_source']['payload']['dst-rse']
        pair = (source,destination)
        
        if pair not in endpoints_failures.keys():
            endpoints_failures[pair] = 1
        if pair in endpoints_failures.keys():
            endpoints_failures[pair] = endpoints_failures[pair]+1
    
        counter += 1
    
    
    #calculate failure rate for each pair
    sorted_endpoints_failures = sorted(endpoints_failures.items(), key=operator.itemgetter(1))
        
    ##and item[1] >= file_thresh    
        
    remove = []    
    for item in sorted_endpoints_failures:
        if item[0] in endpoints_transfers  and item[1]/(endpoints_transfers[item[0]]+item[1]) >= failure_rate:
            #check here to see if pair is in list of already known failures. if it isn't add it, and some kind of time stamp
            #if it is in list, check how long it has been in the list. if threshold time is reached, remove it and send an alarm
            if item[0] not in observed_endpoints.keys():
                observed_endpoints[item[0]] = loop_time
            elif item[0] in observed_endpoints.keys():
                if loop_time - observed_endpoints[item[0]] >= persist_thresh:
                    print("Alert! ", item[0], " at time ",loop_time)
                    alarms += 1
                    del observed_endpoints[item[0]]
           
        
        ##item[1] <= file_thresh or
        ##remove elements of observed_endpoints if they did not satisfy thresholds     
        if item[0] in endpoints_transfers and ( item[1]/(endpoints_transfers[item[0]]+item[1]) <= failure_rate):
            if item[0] in observed_endpoints.keys():
                #print("hello")
                remove.append(item[0])
    
    for key in remove:
        del observed_endpoints[key]
    
    ##
    #remove elements of observed_endpoints if they did not satisfy thresholds
    ##for key in observed_endpoints.keys():
    ##    if not key in endpoints_failures.keys() and endpoints_failures[key] >= file_thresh and endpoints_failures[key]/(endpoints_transfers[key]+endpoints_failures[key]) >= failure_rate :
    ##        remove.append(key)
                
    ##for key in remove:
    ##    del observed_endpoints[key]
            #print("alert")
            #print(item[0],item[1]/pairs[item[0]])
            
            #ratios[item[0]] = item[1]/(endpoints_transfers[item[0]]+item[1])
     
    

    # record pairs of failing sites that are meeting the threshold requirements
     
    # at the end of the loop, loop over the pairs, update list. remove links that have gone away. have a check to see i a pair has been in the failure list for long enough.
    # if requirement is met, send an alert.
    
    #as a start, simply print the word "alert" and increase a counter

#print(observed_endpoints)
    
   
print("time spent: ", (time.time() - time_tracker), "seconds.")
print("alarms: ", alarms)
print("Done.")

Starting loop.
Alert!  ('CERN-PROD_DERIVED', 'UKI-SCOTGRID-DURHAM_DATADISK')  at time  1486130400000
time spent:  693.5750691890717 seconds.
alarms:  1
Done.


In [76]:
#output block:

#print number of alerts made (number of elements in alerts list)
#print alerts



#do the same links trigger multiple alerts. if so, maybe some action needs to be taken to suppress this?
# end goal: vary the thresholds and see how the number of alerts behave. compare to number of tickets filed in this period?

##### 